## Topic: Matching of Job Openings with Skill Database usuing Machine Learning


In [214]:
pip install nltk

Note: you may need to restart the kernel to use updated packages.


In [215]:
pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [216]:
pip install matplotlib

Note: you may need to restart the kernel to use updated packages.


In [217]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import *
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.naive_bayes import GaussianNB
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import LeaveOneOut
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import joblib

In [218]:
# from google.colab import drive
# drive.mount('/content/drive')

In [219]:
skills=pd.read_csv("skill_dataset.csv")

In [220]:
display(skills)

,Job Title,Required Skills,Responsibilities
0,Software Developer,"Proficiency in Java, C++, problem-solving, tea...","- Develop, test, and deploy software applicati..."
1,Graphic Designer,"Adobe Creative Suite, typography, visual commu...",- Create visual content for marketing material...
2,Financial Analyst,"Financial modeling, Excel, data analysis, repo...",- Analyze financial data and trends to provide...
3,Registered Nurse,"Nursing degree, patient care, medical knowledg...",- Provide patient care and administer medicati...
4,Marketing Specialist,"Social media marketing, content creation, anal...",- Plan and execute social media campaigns.<br>...
...,...,...,...
127,Content Strategist,"Content planning, brand messaging",- Develop content strategies to align with bra...
128,Mechanical Maintenance Technician,"Equipment maintenance, repairs",- Maintain and repair mechanical equipment and...
129,Occupational Health Nurse,"Occupational health, health assessments",- Provide occupational health services and wel...
130,Market Researcher,"Market analysis, data collection, consumer ins...",- Conduct market research to gather consumer i...


In [221]:
skills.rename(columns={'Required Skills	': 'Required Skills'}, inplace=True)

## Data Preprocessing

### Remove HTML tags

In [222]:
def remove_html_tags(text):
    import re
    clean = re.compile('<.*?>')
    return re.sub(clean, '', text)

In [223]:
skills['Responsibilities'] = skills['Responsibilities'].apply(remove_html_tags)

### Removing '-' from the begining

In [224]:
omit = '-'
skills['Responsibilities'] = skills['Responsibilities'].str.replace(omit, '')

### Stopword Removal

In [225]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to C:\Users\Anish Kumar
[nltk_data]     K\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\Anish Kumar
[nltk_data]     K\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Anish Kumar
[nltk_data]     K\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [226]:
stop_words = set(stopwords.words('english')) #collecting a list of english stopwords

In [227]:
def remove_stopwords(text):
    tokens = word_tokenize(text)
    filtered_tokens = [word for word in tokens if word.lower() not in stop_words]
    return ' '.join(filtered_tokens)

In [228]:
def lemmatize(x):
  return [lemmatizer.lemmatize(word) for word in x]

In [229]:
skills['Required Skills'] = skills['Required Skills'].apply(remove_stopwords)

In [230]:
display(skills)

,Job Title,Required Skills,Responsibilities
0,Software Developer,"Proficiency Java , C++ , problem-solving , tea...","Develop, test, and deploy software applicatio..."
1,Graphic Designer,"Adobe Creative Suite , typography , visual com...",Create visual content for marketing materials...
2,Financial Analyst,"Financial modeling , Excel , data analysis , r...",Analyze financial data and trends to provide ...
3,Registered Nurse,"Nursing degree , patient care , medical knowle...",Provide patient care and administer medicatio...
4,Marketing Specialist,"Social media marketing , content creation , an...",Plan and execute social media campaigns. Crea...
...,...,...,...
127,Content Strategist,"Content planning , brand messaging",Develop content strategies to align with bran...
128,Mechanical Maintenance Technician,"Equipment maintenance , repairs",Maintain and repair mechanical equipment and ...
129,Occupational Health Nurse,"Occupational health , health assessments",Provide occupational health services and well...
130,Market Researcher,"Market analysis , data collection , consumer i...",Conduct market research to gather consumer in...


In [231]:
skills['skill_list']=skills['Required Skills'].apply(lambda x:[skill.strip() for skill in x.split(',')])
display(skills)

,Job Title,Required Skills,Responsibilities,skill_list
0,Software Developer,"Proficiency Java , C++ , problem-solving , tea...","Develop, test, and deploy software applicatio...","[Proficiency Java, C++, problem-solving, teamw..."
1,Graphic Designer,"Adobe Creative Suite , typography , visual com...",Create visual content for marketing materials...,"[Adobe Creative Suite, typography, visual comm..."
2,Financial Analyst,"Financial modeling , Excel , data analysis , r...",Analyze financial data and trends to provide ...,"[Financial modeling, Excel, data analysis, rep..."
3,Registered Nurse,"Nursing degree , patient care , medical knowle...",Provide patient care and administer medicatio...,"[Nursing degree, patient care, medical knowled..."
4,Marketing Specialist,"Social media marketing , content creation , an...",Plan and execute social media campaigns. Crea...,"[Social media marketing, content creation, ana..."
...,...,...,...,...
127,Content Strategist,"Content planning , brand messaging",Develop content strategies to align with bran...,"[Content planning, brand messaging]"
128,Mechanical Maintenance Technician,"Equipment maintenance , repairs",Maintain and repair mechanical equipment and ...,"[Equipment maintenance, repairs]"
129,Occupational Health Nurse,"Occupational health , health assessments",Provide occupational health services and well...,"[Occupational health, health assessments]"
130,Market Researcher,"Market analysis , data collection , consumer i...",Conduct market research to gather consumer in...,"[Market analysis, data collection, consumer in..."


In [232]:
slist = skills['skill_list'].tolist()
display(slist)

[['Proficiency Java', 'C++', 'problem-solving', 'teamwork'],
 ['Adobe Creative Suite', 'typography', 'visual communication'],
 ['Financial modeling', 'Excel', 'data analysis', 'reporting'],
 ['Nursing degree', 'patient care', 'medical knowledge', 'empathy'],
 ['Social media marketing', 'content creation', 'analytics'],
 ['Python', 'data preprocessing', 'machine learning algorithms'],
 ['User-centered design', 'wireframing', 'prototyping'],
 ['Communication', 'CRM', 'negotiation', 'sales techniques'],
 ['HTML', 'CSS', 'JavaScript', 'responsive design'],
 ['Recruitment', 'employee relations', 'HR policies'],
 ['Project planning', 'communication', 'leadership'],
 ['Java', 'software development', 'debugging'],
 ['Copywriting', 'SEO', 'research', 'storytelling'],
 ['Network configuration', 'troubleshooting', 'security'],
 ['Accounting principles', 'financial reporting', 'Excel'],
 ['Communication', 'problem-solving', 'empathy'],
 ['Architectural design', 'AutoCAD', 'project management'],
 [

In [233]:
combined=[]
for i in slist:
  combined.extend(i)

print(list(combined))

#eliminating duplicates
print(list(set(combined)))

['Proficiency Java', 'C++', 'problem-solving', 'teamwork', 'Adobe Creative Suite', 'typography', 'visual communication', 'Financial modeling', 'Excel', 'data analysis', 'reporting', 'Nursing degree', 'patient care', 'medical knowledge', 'empathy', 'Social media marketing', 'content creation', 'analytics', 'Python', 'data preprocessing', 'machine learning algorithms', 'User-centered design', 'wireframing', 'prototyping', 'Communication', 'CRM', 'negotiation', 'sales techniques', 'HTML', 'CSS', 'JavaScript', 'responsive design', 'Recruitment', 'employee relations', 'HR policies', 'Project planning', 'communication', 'leadership', 'Java', 'software development', 'debugging', 'Copywriting', 'SEO', 'research', 'storytelling', 'Network configuration', 'troubleshooting', 'security', 'Accounting principles', 'financial reporting', 'Excel', 'Communication', 'problem-solving', 'empathy', 'Architectural design', 'AutoCAD', 'project management', 'SQL', 'data analysis', 'data visualization', 'Culin

## Modelling

### Vectorization

In [234]:
vectorizer = TfidfVectorizer(max_features=5)
vectors = vectorizer.fit_transform(skills['Required Skills'])
print("Vocabulary: ", vectorizer.vocabulary_)
X=vectors
# print(X.toarray())
print(X)

# Save the trained vocabulary to a file
joblib.dump(vectorizer.vocabulary_, 'tfidf_vocabulary.pkl')

Vocabulary:  {'data': 1, 'analysis': 0, 'patient': 4, 'design': 2, 'management': 3}
  (2, 0)	0.6586286478914134
  (2, 1)	0.7524681416357296
  (3, 4)	1.0
  (5, 1)	1.0
  (6, 2)	1.0
  (8, 2)	1.0
  (16, 3)	0.6947839741953812
  (16, 2)	0.7192184850247327
  (17, 0)	0.400930738863647
  (17, 1)	0.9161083683900338
  (19, 3)	1.0
  (20, 3)	1.0
  (21, 2)	0.9004618840058723
  (21, 0)	0.43493493243541015
  (22, 4)	1.0
  (24, 2)	1.0
  (25, 3)	1.0
  (27, 4)	1.0
  (29, 3)	1.0
  (31, 4)	1.0
  (34, 3)	0.7071067811865476
  (34, 0)	0.7071067811865476
  (36, 2)	1.0
  (37, 4)	1.0
  (39, 4)	1.0
  :	:
  (85, 1)	0.7524681416357296
  (86, 0)	1.0
  (91, 0)	0.6586286478914134
  (91, 1)	0.7524681416357296
  (92, 3)	1.0
  (94, 3)	0.6947839741953812
  (94, 2)	0.7192184850247327
  (97, 4)	1.0
  (99, 3)	1.0
  (108, 3)	1.0
  (109, 1)	1.0
  (111, 4)	1.0
  (112, 0)	0.6586286478914134
  (112, 1)	0.7524681416357296
  (115, 2)	1.0
  (117, 0)	1.0
  (119, 3)	1.0
  (123, 0)	0.6586286478914134
  (123, 1)	0.7524681416357296
  (12

['tfidf_vocabulary.pkl']

### Train Test Split


In [235]:
Y=skills['Job Title'].values

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.1)

print(X_train,X_test)
print(y_train,y_test)

  (0, 0)	1.0
  (1, 1)	1.0
  (2, 1)	1.0
  (3, 0)	0.6586286478914134
  (3, 1)	0.7524681416357296
  (5, 4)	1.0
  (7, 4)	1.0
  (9, 2)	1.0
  (10, 0)	1.0
  (13, 3)	1.0
  (14, 2)	1.0
  (17, 3)	1.0
  (18, 0)	0.6586286478914134
  (18, 1)	0.7524681416357296
  (22, 3)	0.6586286478914134
  (22, 1)	0.7524681416357296
  (23, 0)	0.400930738863647
  (23, 1)	0.9161083683900338
  (25, 4)	1.0
  (29, 0)	1.0
  (30, 0)	1.0
  (32, 2)	1.0
  (33, 2)	1.0
  (34, 4)	1.0
  (36, 2)	1.0
  :	:
  (75, 0)	0.6586286478914134
  (75, 1)	0.7524681416357296
  (79, 0)	0.6586286478914134
  (79, 1)	0.7524681416357296
  (83, 4)	1.0
  (87, 4)	1.0
  (91, 3)	0.6947839741953812
  (91, 2)	0.7192184850247327
  (92, 3)	1.0
  (94, 2)	1.0
  (97, 4)	1.0
  (98, 2)	1.0
  (100, 0)	1.0
  (102, 4)	1.0
  (105, 2)	0.9004618840058723
  (105, 0)	0.43493493243541015
  (106, 2)	1.0
  (107, 2)	1.0
  (108, 2)	1.0
  (110, 4)	1.0
  (113, 0)	0.6586286478914134
  (113, 1)	0.7524681416357296
  (114, 0)	0.6586286478914134
  (114, 1)	0.7524681416357296
  (1

In [236]:
dct=dict()

### Support Vector Machine

In [237]:
svc = SVC(kernel = 'linear',gamma='auto')
svc.fit(X_train, y_train)
y_pred = svc.predict(X_test)

In [238]:
print(X_test.shape)

(14, 5)


In [239]:
print(classification_report(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))
dct["SVM"]=round(accuracy_score(y_test,y_pred)*100,2)

                              precision    recall  f1-score   support

                        Chef       0.00      0.00      0.00       1.0
     Chief Financial Officer       0.00      0.00      0.00       1.0
           Financial Analyst       0.00      0.00      0.00       1.0
             Fitness Trainer       0.00      0.00      0.00       1.0
               HR Specialist       0.00      0.00      0.00       1.0
          IT Project Manager       0.00      0.00      0.00       0.0
         IT Security Analyst       0.00      0.00      0.00       1.0
             Legal Assistant       0.00      0.00      0.00       1.0
           Marketing Manager       0.00      0.00      0.00       1.0
          Operations Analyst       0.00      0.00      0.00       0.0
          Operations Manager       0.00      0.00      0.00       1.0
                  Pharmacist       0.00      0.00      0.00       1.0
         Pharmacy Technician       0.00      0.00      0.00       0.0
Quality Assurance S

c:\Users\Anish Kumar K\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Anish Kumar K\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Anish Kumar K\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _w

### Naive Bayes

In [240]:
gnb = GaussianNB()
classifier = gnb.fit(X_train.toarray(), y_train)
y_pred = classifier.predict(X_test.toarray())

print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test,y_pred))
dct["Naive Bayes"]=round(accuracy_score(y_test,y_pred)*100,2)

                              precision    recall  f1-score   support

       Architectural Drafter       0.00      0.00      0.00       0.0
                        Chef       0.00      0.00      0.00       1.0
                     Chemist       0.00      0.00      0.00       0.0
     Chief Financial Officer       0.00      0.00      0.00       1.0
            Dental Assistant       0.00      0.00      0.00       0.0
           Financial Analyst       0.00      0.00      0.00       1.0
             Fitness Trainer       0.00      0.00      0.00       1.0
               HR Specialist       0.00      0.00      0.00       1.0
    Healthcare Administrator       0.00      0.00      0.00       0.0
         IT Security Analyst       0.00      0.00      0.00       1.0
             Legal Assistant       0.00      0.00      0.00       1.0
           Marketing Manager       0.00      0.00      0.00       1.0
          Operations Manager       0.00      0.00      0.00       1.0
                  P

c:\Users\Anish Kumar K\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Anish Kumar K\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Anish Kumar K\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _w

In [241]:
bnb = BernoulliNB()
classifier = bnb.fit(X_train.toarray(), y_train)
y_pred = classifier.predict(X_test.toarray())

print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test,y_pred))

c:\Users\Anish Kumar K\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Anish Kumar K\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                              precision    recall  f1-score   support

       Architectural Drafter       0.00      0.00      0.00       0.0
                        Chef       0.00      0.00      0.00       1.0
                     Chemist       0.00      0.00      0.00       0.0
     Chief Financial Officer       0.00      0.00      0.00       1.0
           Financial Analyst       0.00      0.00      0.00       1.0
             Fitness Trainer       0.00      0.00      0.00       1.0
               HR Specialist       0.00      0.00      0.00       1.0
    Healthcare Administrator       0.00      0.00      0.00       0.0
         IT Security Analyst       0.00      0.00      0.00       1.0
             Legal Assistant       0.00      0.00      0.00       1.0
           Marketing Manager       0.00      0.00      0.00       1.0
          Operations Manager       0.00      0.00      0.00       1.0
                  Pharmacist       0.00      0.00      0.00       1.0
Quality Assurance S

c:\Users\Anish Kumar K\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Anish Kumar K\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Anish Kumar K\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _w

### K Nearest Neighbours

In [242]:
# Standardize the features
scaler = StandardScaler()
# X_train_scaled = scaler.fit_transform(X_train)
# X_test_scaled = scaler.transform(X_test)

# Initialize the KNN classifier
knn = KNeighborsClassifier(n_neighbors=3)  # You can adjust n_neighbors

# Train the model
knn.fit(X_train, y_train)

# Make predictions on the test data
predictions = knn.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, predictions)
print("Accuracy:", accuracy)
dct["KNN"]=round(accuracy_score(y_test,predictions)*100,2)

Accuracy: 0.0


### Random Forest

In [243]:
classifier = RandomForestClassifier(n_estimators=1000, random_state=0)
classifier.fit(X_train,y_train)

RandomForestClassifier(n_estimators=1000, random_state=0)

In [244]:
predictions = classifier.predict(X_test)
# Calculate accuracy
accuracy = accuracy_score(y_test, predictions)
print("Accuracy:", accuracy)
dct["Random Forest"]=round(accuracy_score(y_test,predictions)*100,2)

Accuracy: 0.0


In [245]:
print(dct.keys())

dict_keys(['SVM', 'Naive Bayes', 'KNN', 'Random Forest'])


In [246]:
max_value = max(dct.values())
max_key = [key for key, value in dct.items() if value == max_value][0]
print("Key for max value:",max_key)

Key for max value: SVM


In [247]:
if max_key=='SVM':
  joblib.dump(svc, 'resume_matching_model.pkl')
if max_key=='Naive Bayes':
  joblib.dump(gnb, 'resume_matching_model.pkl')
if max_key=='KNN':
  joblib.dump(knn, 'resume_matching_model.pkl')
if max_key=='Random Forest':
  joblib.dump(classifier, 'resume_matching_model.pkl')